In [1]:
-- Load utility scripts (e.g. from from ugcore/scripts)
ug_load_script("ug_util.lua")
ug_load_script("util/refinement_util.lua")

-- Initialize with the world dimension dim=2 and the algebra type
blockSize = 2 -- select 2 for 2x2 point-block system
InitUG(2, AlgebraType("CPU", blockSize));  

* Initializing: paths... done, bridge... done, plugins... done                 *
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/d3f/lua/lua-include.lua
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/Limex/lua/lua-include.lua
LIMEX-Plugin: Loading util.limex.*
Loading Plugin Lua Content from /Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/lua-include.lua
RepoTrend-Plugin: Extended package.path:./?.lua;/usr/local/share/lua/5.1/?.lua;/usr/local/share/lua/5.1/?/init.lua;/usr/local/lib/lua/5.1/?.lua;/usr/local/lib/lua/5.1/?/init.lua;/Users/anaegel/Software/ug4-git/plugins/RepoTrend/lua/?.lua


In [2]:
-- Parse parameters and print help
gridName= "grids/laplace_sample_grid_2d.ugx" 
numRefs	= 4

-- Load a domain without initial refinements.
mandatorySubsets = {"Inner", "Boundary"}
dom = util.CreateDomain(gridName, 0, mandatorySubsets)

-- Refine the domain (redistribution is handled internally for parallel runs)

util.refinement.CreateRegularHierarchy(dom, numRefs, true)

Loading Domain grids/laplace_sample_grid_2d.ugx ... done.
Performing integrity check on domain ... done.

util.refinement: - refining level 0
util.refinement: - refining level 1
util.refinement: - refining level 2
util.refinement: - refining level 3


## B) Ansatzraum


In [3]:
approxSpace = ApproximationSpace(dom)
-- TODO: Add functions
approxSpace:add_fct("u", "Lagrange", 1)   -- lineare Ansatzfunktionen
approxSpace:add_fct("c", "Lagrange", 1)

approxSpace:init_levels()
approxSpace:init_top_surface()
approxSpace:print_statistic()

| ---------------------------------------------------------------------------- |
|  Number of DoFs (All Procs)                                                  |
|  Algebra: Block 2 (divide by 2 for #Index)                                   |
|                                                                              |
|    GridLevel   |       Domain |     0: Inner |  1: Boundary                  |
| ---------------------------------------------------------------------------- |
| (lev,    0)    |           18 |            2 |           16 |
| (lev,    1)    |           50 |           18 |           32 |
| (lev,    2)    |          162 |           98 |           64 |
| (lev,    3)    |          578 |          450 |          128 |
| (lev,    4)    |         2178 |         1922 |          256 |
| (lev,    0, g) |           18 |            2 |           16 |
| (lev,    1, g) |           50 |           18 |           32 |
| (lev,    2, g) |          162 |           98 |           64 |
| 

## C) Elementdiskretisierung (FV)

$$ \frac{\partial c}{ \partial t} + \nabla \cdot [- M \nabla \mu]  = 0$$ 
$$ \nabla \cdot [- \lambda \nabla c] + F'(c) = \mu$$ 

In [4]:
M=1
lambda=0.01


charLength = 1.0
charTime = charLength*charLength/M

In [5]:
A = 100.0

function HelmholtzEnergyF0(c) 
  return A*c*c*(1.0-c)*(1.0-c)  
end

-- TODO: implement F'(c)
function HelmholtzEnergyF1(c) 
  return 2.0*A*c*(2*c*c - 3*c + 1)   
end

-- TODO: implement F''(c)
function HelmholtzEnergyF2(c) 
   return 2.0*A*(6.0*(c-1)*c + 1)    
end


In [6]:
elemDisc = {}
elemDisc["u"] = ConvectionDiffusion("u", "Inner", "fv1")  -- potential
elemDisc["c"] = ConvectionDiffusion("c", "Inner", "fv1")  -- gating variable 

$$ \frac{\partial c}{ \partial t} + \nabla \cdot [- M \nabla \mu]  = 0$$ 

In [7]:
elemDisc["u"]:set_mass_scale(0.0)  
elemDisc["u"]:set_mass(elemDisc["c"]:value()) 
elemDisc["u"]:set_diffusion(M)
elemDisc["u"]:set_reaction(0.0)  
elemDisc["u"]:set_source(0.0)

$$ \nabla \cdot [- \lambda \nabla c] + F'(c) = \mu$$ 

In [8]:
dFdc = LuaUserFunctionNumber("HelmholtzEnergyF1", 1)
dFdc:set_input(0, elemDisc["c"]:value())
dFdc:set_deriv(0, "HelmholtzEnergyF2")

In [9]:
elemDisc["c"]:set_stationary()
elemDisc["c"]:set_diffusion(lambda)
elemDisc["c"]:set_reaction(dFdc)          
elemDisc["c"]:set_source(elemDisc["u"]:value())  

## D) Anfangswerte

In [24]:
-- Initial values ("Anfangswerte")
function MyInitialValueU(x, y)
  return 0.63 + 0.02*(math.random()-0.5)
end

function MyInitialValueC(x, y)
  return 0.0 
end


## E) Randwerte

In [25]:
dirichletBND = DirichletBoundary()
-- dirichletBND:add(1.0, "u", "Boundary")
--dirichletBND:add(1.0, "c", "Boundary")

## F) Diskretisierung auf ganzem Gebiet

In [26]:
domainDisc = DomainDiscretization(approxSpace)
domainDisc:add(elemDisc["u"])
domainDisc:add(elemDisc["c"])


In [27]:
dbgWriter=GridFunctionDebugWriter(approxSpace)

## G) Aufsetzen des Lösers (using 'util/solver_util.lua')

In [31]:
local solverDesc = {
	
	-- Newton's method for non-linear problem
	type = "newton",

  -- BiCGStab 
	linSolver = {
	 type = "bicgstab",
	
	 precond = {
		  type		= "gmg",
		  approxSpace	= approxSpace,
		  smoother	= "sgs",
		  baseSolver	= "lu"
	 },
        
    
	
  },
	
}

nlsolver = util.solver.CreateSolver(solverDesc)
-- nlsolver:set_debug(dbgWriter)
print (nlsolver)

[ NewtonSolverCPU2: 0x7fe4ed47fdd0


## H) Lösen

In [35]:
local u = GridFunction(approxSpace)
u:set(0.0)

Interpolate("MyInitialValueU", u, "u")
Interpolate("MyInitialValueC", u, "c")

local startTime = 0.0
local endTime = 1e-3*charTime
local dt = (endTime-startTime)/200.0
util.SolveNonlinearTimeProblem(u, domainDisc, nlsolver, VTKOutput(),
"vtk/CahnHilliard", "Theta", 0.5, startTime, endTime, dt, dt*1e-4);

print("done")


SolveNonlinearTimeProblem, Newton Solver setup:
NewtonSolver
 LinearSolver: 
 | BiCGStab( restart = 0, min_orthogonality = 0)
 |  Convergence Check: StdConvCheck( max steps = 100, min defect = 1e-12, relative reduction = 1e-06)
 |  Preconditioner: 
 |  # GeometricMultigrid (V-Cycle)
 |  #  Smoother (3x pre, 3x post): Symmetric Gauss-Seidel( damping = ConstantDamping(1))
 |  #  Basesolver ( Baselevel = 0, gathered base = false): AgglomeratingSolver: SuperLU
 ConvergenceCheck: StdConvCheck( max steps = 100, min defect = 1e-12, relative reduction = 1e-06)
 LineSearch:  not set.

>> Writing start values
++++++ TIMESTEP 1 BEGIN (current time: 0) ++++++
++++++ Time step size: 5e-06

   # ########        Newton Solver        #################
   # ########  (Linear Solver: BiCGStab)  #################
   #   Iter      Defect         Rate 
   #    0:    1.938264e-02      -------

      % %%%%%%%%             BiCGStab              %%%%%%%%%%%
      % %%%%%%%%   (Precond: Geometric MultiGrid)  %